In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import ast
import torch.nn as nn

**训练集(用于训练和更新参数)300个样本**

In [7]:
data_source = pd.read_excel('dataset.xlsx')
data_source = data_source.iloc[1:]
trajorites = np.array(data_source.iloc[:,8].values)
processed_trajorites = []
train_target = []
for i in range(trajorites.shape[0]):
    current_trajoritey = ast.literal_eval(trajorites[i])
    if len(current_trajoritey)>=14:
        processed_trajorites.append(current_trajoritey[:7])
        train_target.append(current_trajoritey[7:14])
processed_trajorites = torch.tensor(processed_trajorites)[:300,:]
train_target = torch.tensor(train_target)[:300,:]
print(processed_trajorites.shape,train_target.shape)

torch.Size([300, 7, 2]) torch.Size([300, 7, 2])


**验证集(用于保存表现最好的模型)300个另外的样本**

In [21]:
valdata = pd.read_excel('valset.xlsx')
valdata = valdata.iloc[1:]
val_trajorites = np.array(valdata.iloc[:,8].values)
val_input_all = []
val_target_all = []
for j in range(val_trajorites.shape[0]):
    current_trajoritey_val = ast.literal_eval(val_trajorites[j])
    if len(current_trajoritey_val)>=14:
        val_input_all.append(current_trajoritey_val[:7])
        val_target_all.append(current_trajoritey_val[7:14])
val_input = torch.tensor(val_input_all)[:300,:]
val_target = torch.tensor(val_target_all)[:300,:]
print(val_input.shape,val_target.shape)

torch.Size([300, 7, 2]) torch.Size([300, 7, 2])


In [5]:
class TrajectoryDataset(Dataset):
    def __init__(self, trajectory_tensor):
        self.trajectory_tensor = trajectory_tensor

    def __len__(self):
        return len(self.trajectory_tensor)

    def __getitem__(self, index):
        return self.trajectory_tensor[index]

In [16]:
batch_size = 32
dataset = TrajectoryDataset(processed_trajorites)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, num_layers, hidden_size, num_heads):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)  # 嵌入层
        x = x.permute(1, 0, 2)  # 调整维度顺序
        x = self.transformer(x, x)  # Transformer模型的前向传播
        x = x.permute(1, 0, 2)  # 调整维度顺序
        x = self.fc(x)  # 全连接层
        return x

input_size = 2  # 输入特征维度
output_size = 2  # 输出特征维度
num_layers = 2  # Transformer编码器和解码器的层数
hidden_size = 64  # Transformer内部隐藏层维度
num_heads = 4  # 多头注意力机制中的头数

model = TransformerModel(input_size, output_size, num_layers, hidden_size, num_heads)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


best_val_loss = 1e10

for epoch in range(2000):
    print(f"=============epoch={epoch+1}=============")    
    train_inputs = processed_trajorites.permute(1, 0, 2)
    val_inputs = val_input.permute(1, 0, 2)
    train_targets = train_target.permute(1, 0, 2)
    val_targets = val_target.permute(1, 0, 2)
    optimizer.zero_grad()
    train_outputs = model(train_inputs)
    train_loss = criterion(train_outputs, train_targets)
    val_ouputs = model(val_inputs)
    val_loss = criterion(val_ouputs,val_targets)
    print(f'Train loss (MSE)={train_loss}   Val losss (MSE)={val_loss}')
    if val_loss<best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(),'best_model.pt')
        print("model saving...")
    train_loss.backward()
    optimizer.step()


=============epoch=1=============
Train loss (MSE)=930.1218872070312   Val losss (MSE)=930.2617797851562
model saving...
=============epoch=2=============
Train loss (MSE)=854.15087890625   Val losss (MSE)=855.2777099609375
model saving...
=============epoch=3=============
Train loss (MSE)=805.3062133789062   Val losss (MSE)=805.2669677734375
model saving...
=============epoch=4=============
Train loss (MSE)=780.1233520507812   Val losss (MSE)=780.3232421875
model saving...
=============epoch=5=============
Train loss (MSE)=764.044189453125   Val losss (MSE)=764.1315307617188
model saving...
=============epoch=6=============
Train loss (MSE)=751.5540771484375   Val losss (MSE)=751.3853759765625
model saving...
=============epoch=7=============
Train loss (MSE)=741.1471557617188   Val losss (MSE)=741.0101318359375
model saving...
=============epoch=8=============
Train loss (MSE)=732.3619995117188   Val losss (MSE)=732.3649291992188
model saving...
=============epoch=9=============
Trai

**测试集(用于检验模型)1949个样本**

In [24]:
test_input = torch.tensor(val_input_all)[300:,:]
print(test_input.shape)
test_target = torch.tensor(val_target_all)[300:,:]
test_inputs = test_input.permute(1, 0, 2)
test_targets = test_target.permute(1, 0, 2)
test_output = model(test_inputs)
print(f'Test loss is :{criterion(test_output,test_targets)}')

torch.Size([1949, 7, 2])
Test loss is :0.0037290523760020733


**======================模型训练，验证和测试完成，进入直接应用======================**